<a href="https://colab.research.google.com/github/wanderloop/WanderlustAI/blob/master/midpoint_to_coo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pandas import DataFrame, read_csv, concat, set_option
set_option('display.max_columns', None)
set_option('display.max_rows', None)
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
!pip install dtale
import dtale
import dtale.app as dtale_app
from numpy import nan, reshape, mean, std, int16, float32
from plotly.figure_factory import create_table
!pip install tpot
from tpot import TPOTRegressor
from sklearn.impute import KNNImputer
from sklearn.pipeline import make_pipeline
!pip install shap
from shap import initjs, TreeExplainer, summary_plot, force_plot
!pip install catboost
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from scipy.stats import t, skewtest
!pip install researchpy
from researchpy import summary_cont
from gc import collect
from joblib import dump
from IPython.display import clear_output
clear_output()

In [ ]:
!pip list | grep numpy
!pip list | grep pandas
!pip list | grep dtale
!pip list | grep plotly
!pip list | grep scikit-learn
!pip list | grep scipy
!pip list | grep researchpy
!pip list | grep ipython

numpy                         1.18.5         
pandas                        1.1.2          
pandas-datareader             0.9.0          
pandas-gbq                    0.13.2         
pandas-profiling              1.4.1          
sklearn-pandas                1.8.0          
dtale                         1.16.0         
plotly                        4.4.1          
scikit-learn                  0.22.2.post1   
scipy                         1.4.1          
researchpy                    0.2.3          
ipython                       5.5.0          
ipython-genutils              0.2.0          
ipython-sql                   0.3.9          


In [ ]:
df = read_csv('https://raw.githubusercontent.com/wanderloop/WanderlustAI/master/assumed_PHA.csv',
              
              dtype={'X1': 'int16',
                     'Y1': 'int16',
                     'X2': 'int16',
                     'Y2': 'int16',
                     'Area': 'int16',
                     'long': 'float32',
                     'lat': 'float32',},
              
              low_memory=True,
              usecols='X1 Y1 X2 Y2 lat'.split())
             
data = create_table(df.head())
data.show()

In [ ]:
# Creating new feature: MPX
df['MPX'] = (df['X1'] + df['X2']) / 2
df['MPY'] = (df['Y1'] + df['Y2']) / 2
df = df.drop(['X1', 'X2', 'Y1', 'Y2'],
             axis=1,)

df['MPX'] = df['MPX'].astype('float32')
df['MPY'] = df['MPY'].astype('float32')
data = create_table(df.head())
data.show()

In [ ]:
del data
collect() # Garbage collection

19091

In [ ]:
tail = create_table(df.tail())
tail.show()

In [ ]:
del tail
collect()

19318

In [ ]:
df1, df2 = df.head(), df.tail()
conc_data_row = concat([df1,
                        df2,],
                        axis=0, # Row-wise operation
                        join='outer',) # Returns both dataframes' columns

conc_data_row = create_table(conc_data_row)
conc_data_row.show()

In [ ]:
del conc_data_row
collect()

22300

In [ ]:
assert df['MPX'].notnull().all()
assert df['MPY'].notnull().all()
assert df['lat'].notnull().all()

In [ ]:
# Dropping columns with more than 50% missing values
df = df.dropna(thresh=df.shape[0]*0.5,
               how='all', # Only drop columns when all values are NA
               axis='columns',)
df.shape

(18, 3)

In [ ]:
df.info(verbose=True,
        memory_usage='deep',) # Actual memory usage calculation

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lat     18 non-null     float32
 1   MPX     18 non-null     float32
 2   MPY     18 non-null     float32
dtypes: float32(3)
memory usage: 344.0 bytes


In [ ]:
dtale_app.USE_COLAB = True
report = dtale.show(df,
                    ignore_duplicate=True,)
report

https://9ueuc4o9sro-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/1

In [ ]:
# Calculating t-statistic
sample_size = 18
print(t.ppf(1-0.025, sample_size - 1))

2.1098155778331806


In [ ]:
# Checking the distribution of the target variable
DataToTest = df['lat']
stat, p = skewtest(DataToTest)
print(f'stat={stat}', 
      f'p={p}')

if p > 0.001:
    print('Normal distribution')
else:
    print('Not a normal distribution')

stat=0.022829372422934105 p=0.9817863783212984
Normal distribution


In [ ]:
summary = summary_cont(df[['MPX',
                           'MPY',
                           'lat',]])

summary = create_table(summary)
summary.show()

In [ ]:
# Define predictor (X) and target variables (y)
X = df[['MPX',
        'MPY',]]
        
y = df['lat'].to_numpy().reshape(-1, 1)

In [ ]:
imp = KNNImputer(missing_values=nan, # Missing values marked as nan
                 n_neighbors=5, # No. of nearby values
                 weights='distance', # Nearest value = Greatest influence
                 metric='nan_euclidean',)

In [ ]:
regr = TPOTRegressor(generations=5, 
                     population_size=100,
                     scoring='neg_root_mean_squared_error',
                     cv=10,
                     n_jobs=-1, # Use all CPU cores
                     random_state=1,
                     verbosity=2,)

In [ ]:
pipeline = make_pipeline(imp, regr)
pipeline.fit(X, y.reshape(-1),)


Generation 1 - Current best internal CV score: -4.966127831431096e-05
Generation 2 - Current best internal CV score: -1.9502802602147202e-05
Generation 3 - Current best internal CV score: -1.9502802602147202e-05
Generation 4 - Current best internal CV score: -1.938024995761485e-05
Generation 5 - Current best internal CV score: -1.895107407882979e-05
Best pipeline: RidgeCV(PolynomialFeatures(DecisionTreeRegressor(input_matrix, max_depth=5, min_samples_leaf=6, min_samples_split=20), degree=2, include_bias=False, interaction_only=False))


Pipeline(memory=None,
         steps=[('knnimputer',
                 KNNImputer(add_indicator=False, copy=True,
                            metric='nan_euclidean', missing_values=nan,
                            n_neighbors=5, weights='distance')),
                ('tpotregressor',
                 TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=10,
                               disable_update_check=False, early_stop=None,
                               generations=5,
                               log_file=<ipykernel.iostream.OutStream object at 0x7f881c039ac8>,
                               max_eval_time_mins=5, max_time_mins=None,
                               memory=None, mutation_rate=0.9, n_jobs=-1,
                               offspring_size=None,
                               periodic_checkpoint_folder=None,
                               population_size=100, random_state=1,
                               scoring='neg_root_mean_squared_error',
                    